# Interpretable Machine Learning
## Exercise Sheet: 4
## This exercise sheet covers chapters 6, 8.1 + 8.2 from the IML book by Christoph Molnar

Kristin Blesch (blesch@leibniz-bips.de)<br>
Niklas Koenen (koenen@leibniz-bips.de)
<hr style="border:1.5px solid gray"> </hr>

# 1) Partial Dependence Plot (PDP)

In this task, the concept of Partial Dependence Plots (PDP) will be explained. For this, you have to train a multilayer perceptron (MLP) on the regression dataset `sklearn.datasets.fetch_california_housing` and then implement the PDP method on your own.

## a) Data
Load the data and get familiar with it by extracting the features $X$ as a `pandas.DataFrame` and the target $Y$. Then, answer the following questions:

- What are the features and what are their types (numeric, binary, categorical)?
- What is the target outcome?

**Solution:**

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing

cal_housing = fetch_california_housing()

# Get features as pd.DataFrame
X = pd.DataFrame(cal_housing.data, columns=cal_housing.feature_names)

# Get target
Y = cal_housing.target

*   The dataset includes the following features
  * `MedInc`: Median income in block group (numerical)
  * `HouseAge`: Median house age in block group (numerical)
  * `AveRooms`: Average number of rooms per household (numerical)
  * `AveBedrms`: Average number of bedrooms per household (numerical)
  * `Population`: Block group population (numerical)
  * `AveOccup`: Average number of household members (numerical)
  * `Latitude`: Block group latitude (numerical)
  * `Longitude`: Block group longitude (numerical)

*   The target variable is the median house value for California districts in hundreds of thousands of dollars ($100,000)

## b) Train MLP
Create training and test data using the function `train_test_split` and train the predefined MLP model. Then calculate the $R^2$ value on the test data using the [score](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html#sklearn.neural_network.MLPRegressor.score) method.

**Hint:** Use function `fit(X_train, y_train)`.

**Solution:**

In [ ]:
# Create train and test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0)

# Define model
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer
from sklearn.neural_network import MLPRegressor

model =  make_pipeline(
    QuantileTransformer(),
    MLPRegressor(
        hidden_layer_sizes=(50, 50), learning_rate_init=0.01, early_stopping=True
    )
)

# Train the model on the trainings data

model.fit(X_train, y_train)

# Calculate the R^2- value

f"Test R2 score: {model.score(X_test, y_test):.2f}"

## c) Implement PDP
**i)** Now let's apply the method to the feature `HouseAge`. Crate a PDP for this feature.

**Hint:** Get the minimum and maximum value of this feature and create a vector with the intermediate values (`np.linspace(min, max, 100)`). Insert these values into the test data in a loop and calculate the average output. Finally, create a plot with `plt.plot`.

**Solution:**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get the domain for feature 'HouseAge'
min_value = min(X_test.HouseAge)
max_value = max(X_test.HouseAge)
HouseAge = np.linspace(min_value, max_value, 100)

# Calculate PDP
y = []
for i in HouseAge:
  x = X_test.copy()
  x.HouseAge = i
  y.append(np.mean(model.predict(x)))

# Plot PDP
plt.plot(HouseAge, y)
plt.show()

**ii)** Create another PDP for this feature with the predefined function [`PartialDependenceDisplay`](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.PartialDependenceDisplay.html#sklearn-inspection-partialdependencedisplay) and check your implementation.

**Solution:**

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

display = PartialDependenceDisplay.from_estimator(
    model,
    X_test,
    features = ["HouseAge"],
    grid_resolution=100,
)

**iii)** What do these plots tell us? Can they indicate the importance of the feature and if not, how do you get it?

**Solution:**

A Partial Dependence Plot describes the influence of a feature on the target on average over the marginal distribution, i.e. every realization in the dataset is considered. In addition, it illustrates the relationship between one feature and the outcome, e.g. is it linear or non-linear? A flat PDP indicates that the feature is less important, and the more the PDP varies, the more important is the feature.

However, a single PDP can not be used to determine the importance of a feature because the variations need to be considered in relation to the other PDPs. This problem solves the following partial dependence-based feature importance measure proposed by Greenwell: For feature $i \in \{1, \ldots, K\}$ and $n$ samples
$$
I_i(x) = \sqrt{\frac{1}{K - 1} \sum_{k=1}^K\left( \hat{f}_i(x) - \frac{1}{K} \sum_{j = 1}^K \hat{f}_{j}(x)\right)^2}\quad \text{with}\quad \hat{f}_i(x) = \frac{1}{n} \sum_{k = 1}^n \hat{f}(x, x_C^{(k)})$$

# 2) Accumulated Local Effects (ALE) Plots

## a) Theory


**i)** What is the problem with PDPs and why can't we blindly trust PDP-based Feature Importance?

**Solution:**

The major problem with PDPs is that they only consider the marginal effect and assume that the features are uncorrelated among each other. When the features are correlated, we create in the calculated mean new data points in areas of the feature distribution where the actual probability is very low. In addition, when considering the marginal effect, values may cancel each other out, even though individually they have a very strong impact.

All of these points ensure that the PDP-based Feature Importance measure is only useful under sufficient assumptions, and otherwise, it can make misleading statements. In addition, it captures only the main effect of the feature and ignores possible feature interactions.

**ii)** Explain the concepts of PDP, M-Plots and ALE plots and their differences. What are the respective advantages and disadvantages of each method compared to the other two?

**Solution:**

**Partial Dependence Plots:** Average the predictions over the marginal feature distribution (marginal expectation)
$$
\hat{f}_{i, \text{PDP}}(x) = \int_{X_C} \hat{f}(x, X_C)\ d\mathbb{P}(X_C)
$$
> “Let me show you what the model predicts on average when each data instance has the value v for that feature. I ignore whether the value v makes sense for all data instances.”
* Pros: 
    - intuitive; It is a very naive approach
    - easy to implement
    - causal interpretation
* Cons:
    - assumption of independence, i.e. it is assumed that the feature for the PDP is uncorrelated with all other features
    - heterogeneous effects might be hidden; it shows the average marginal effect but individual eliminations are not considered
    
**Marginal-Plots (M-Plots):** Average the predictions over the conditional feature distribution, i.e. we average the predictions conditional on each grid value (in a predefined neighborhood) of the feature of interest, instead of assuming the marginal distribution at each grid value.
$$
\hat{f}_{i, \text{M}}(x) = \int_{X_C} \hat{f}(x, X_C)\ d\mathbb{P}(X_C \mid X = x)
$$
> “Let me show you what the model predicts on average for data instances that have values close to v for that feature. The effect could be due to that feature, but also due to correlated features.”
* Pros:
    - avoid averaging predictions of unlikely data instances (conditional distribution instead of marginal distribution)
* Cons:
    - mix the effect of a feature with the effects of all correlated features

**Accumulated Local Effect Plots:** Average the changes in the predictions and accumulate them over the grid
$$
\hat{f}_{i,ALE}(x) =
\int_{z_{0}}^{x}\left(\int_{X_C}\hat{g}_i(z,X_C)\ d\mathbb{P}(X_C|X = z)\right)dz-\text{Const}\quad \text{with}\quad \hat{g}_i(x,x_c)=\frac{\partial\hat{f}_i(x,x_C)}{\partial x}
$$
>“Let me show you how the model predictions change in a small”window" of the feature around v for data instances in that window."
* Pros:
    - unbiased, i.e. they still work when features are correlated
    - faster to compute; $\mathcal{O}(n)$ with $n$ grid points/intervals
    - clear interpretation because ALE plots are centered at zero
* Cons:
    - interpretation of the effect across intervals is not permissible if the features are strongly correlated. The effects are computed per interval (locally) and therefore the interpretation of the effect can only be local.
    - can become a bit shaky with a high number of intervals

# b) Example: ALE Plot

**i)** Install the package [ALEPython](https://github.com/blent-ai/ALEPython) from Github and create an ALE plot for the example from task **1)** as well for the feature `HouseAge`.

**Hint:** See the 'Install' and 'Usage' section on the Github page.

**Solution:**

In [ ]:
# Install package
# To use a code chunk as a console the line must start with '!'.

!pip install git+https://github.com/MaximeJumelle/ALEPython.git@dev#egg=alepython

In [ ]:
# Create the ALE plot (set 'monte_carlo = False')

from alepython import ale_plot
ale_plot(model, X_train, 'HouseAge')
plt.show()

**II)** How does the ALE plot differ from the PDP in task 1? Give possible reasons for the similarities or differences.

**Solution:**

* ALE plots show the relative effect by centering at zero
* We see that newer houses have a positive effect and older ones have a negative effect on the sales price. We could not see that in the PDP because they are not centered, i.e. no subtraction of the average effect.
* In our case, the plots look very similar, which might be due to the very low correlation from this feature to the others.

**Note:** Of course, the plots may look different for other seeds (especially at the edges where there are few data points).